# Hledání nejkratší cesty v bludišti

## Textový popis

Tento projekt se zabývá řešením (hledáním nejkratší cesty) a také
základním generováním bludišť. Základním vstupem bude bludiště
$n\times n$, přičemž vstup do bludiště bude vždy levý horní roh a výstup
bude vždy pravý dolní roh. Z jedné buňky do druhé se lze dostat pouze
přes společnou hranu (nikoliv přes roh). Cílem projektu je implementovat
algoritmy pro načítání, hledání nejkratší cesty a generování bludiště.

Na začátku bude implementována funkce pro načítání bludiště z CSV
souboru. Tato funkce bude umět načítat bludiště o libovolném rozměru
$n\times n$ a uložit ho do paměti v podobě NumPy matice s True/False
hodnotami (True = buňka je neprůchozí). Poté bude implementován
algoritmus pro hledání nejkratší cesty. Poslední částí bude vytvoření
generátoru bludiště za použití algoritmu pro hledání nejkratší cesty.

Výstup bude formou obrázku (černá = neprostupná část, bílá = průchozí,
červená = nejkratší cesta).

## Funcionality

- Implementovat načítání bludiště z CSV souboru
- Implementovat algoritmus pro hledání nejkratší cesty (mezi levým horním rohem a pravým dolním rohem) za použití knihovny NumPy,který bude pracovat v následujících dvou krocích:
  - Sestavení incidenční matice
  - Hledání nejkratší cesty např. pomocí Dijkstrova algoritmu (jsou i jiné možnosti jako hledání do šířky, výběr je na vás)
- Zapsat bludiště a nalezenou cestu do černobílého obrázku, kde cesta bude vyznačena červeně
- Vytvořit funkci pro generování bludiště tak, aby mělo řešení (tj. aby existovala cesta mezi levým horním a pravým dolním rohem)
  - funkce začne s nějakou šablonou (předdefinované v kódu) a poté bude zaplňovat bludiště v náhodných místech a kontrolovat, zda je stále průchozí
  - šablon bude více (např. empty = volné bludiště, slalom = bariéry
    aby cesta musela minimálně mít tvar S, \...) - budou s obrázky ukázané v Readme nebo examples.ipynb

In [63]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import scipy.sparse as sp
from typing import Union, Tuple
import heapq

class Maze:
    def __init__(self, input_data: Union[str, np.ndarray] = None):
        self.map = None
        self.start = None
        self.goal = None
        if input_data is not None:
            if isinstance(input_data, str):
                self.map = self.from_csv(input_data)
            elif isinstance(input_data, np.ndarray):
                self.map = input_data
            self.start = 0
            self.goal = self.map.size - 1

    def from_csv(self, filename: str) -> np.ndarray:
        return np.genfromtxt(filename, delimiter=',', dtype=int).astype(bool)
    
    def incidence_matrix(self) -> sp.lil_matrix:
        def index(r: int, c: int) -> int:
            return r * cols + c

        rows, cols = self.map.shape
        node_count = rows * cols
        incidence = sp.lil_matrix((node_count, node_count), dtype=int)
        
        for r in range(rows):
            for c in range(cols):
                if not self.map[r, c]:
                    current_index = index(r, c)
                    # check right neighbor
                    if c + 1 < cols and not self.map[r, c + 1]:
                        right_index = index(r, c + 1)
                        incidence[current_index, right_index] = 1
                        incidence[right_index, current_index] = 1
                    # check down neighbor
                    if r + 1 < rows and not self.map[r + 1, c]:
                        down_index = index(r + 1, c)
                        incidence[current_index, down_index] = 1
                        incidence[down_index, current_index] = 1

        return incidence


        



In [64]:
maze = Maze("data/maze_3.csv")
print(maze.map)


[[False False False False  True False False False False False  True False
   True False False  True False False False False]
 [False False  True False False False False  True  True False False False
  False False False False False False False False]
 [False  True False  True False  True False  True  True  True False False
  False  True False False  True False False False]
 [False False  True False  True  True  True False  True False False False
  False False False False  True  True False False]
 [False False False  True  True  True  True  True False  True False False
   True False False False  True False False False]
 [ True  True False False  True False  True False False False False False
  False  True False  True  True False False False]
 [ True  True  True False  True False  True False False  True  True False
  False False False  True  True  True False False]
 [ True False False False False  True  True  True False False  True  True
   True False False False  True  True  True False]
